Import the data provided in the `all_states.json` file from your Terminal. Name the database `birth_data` and the collection `states`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d birth_data -c states --drop --jsonArray all_states.json`

In [16]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from api_key import api_key

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [7]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'birth_data', 'config', 'epa', 'local', 'met', 'uk_food']


In [9]:
# assign the uk_food database to a variable name
db = mongo["birth_data"]

In [10]:
# review the collections in our new database
print(db.list_collection_names())

['states']


In [12]:
# review a document in the establishments collection
pprint(db.states.find_one())

{'3P': '0',
 '3PA': '0',
 'AST': '67',
 'BLK': '24',
 'Birthday': 'Feb 8, 1967',
 'City': 'Birmingham',
 'FG': '383',
 'FGA': '746',
 'FT': '260',
 'FTA': '360',
 'GP': '149',
 'MP': '2143',
 'ORB': '311',
 'PF': '284',
 'PTS': '1026',
 'Player': 'Michael Ansley',
 'STL': '51',
 'State': 'AL',
 'TO': '85',
 'TRB': '621',
 'Years': '3',
 '_id': ObjectId('661883863383a053ef184a23')}


In [13]:
# assign the collection to a variable
states = db['states']

Add in Coordinates for each players' birth location

Using Geoapify

In [57]:
# Adding Lat & Lon to the DataBase
db.states.update_many(
    {},
    {
        '$set': {
            'Lat': {'$toDouble': '$Latitude'},  
            'Lon': {'$toDouble': '$Longitude'}  
        }
    },
    upsert=True
)

UpdateResult({'n': 4330, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [44]:
# review a document in the establishments collection
pprint(db.states.find_one())

{'3P': '0',
 '3PA': '0',
 'AST': '67',
 'BLK': '24',
 'Birthday': 'Feb 8, 1967',
 'City': 'Birmingham',
 'FG': '383',
 'FGA': '746',
 'FT': '260',
 'FTA': '360',
 'GP': '149',
 'Lat': {'$toDouble': '$Latitude'},
 'Lon': {'$toDouble': '$Longitude'},
 'MP': '2143',
 'ORB': '311',
 'PF': '284',
 'PTS': '1026',
 'Player': 'Michael Ansley',
 'STL': '51',
 'State': 'AL',
 'TO': '85',
 'TRB': '621',
 'Years': '3',
 '_id': ObjectId('661883863383a053ef184a23')}


In [ ]:
state = states['State']
city = states['City']

In [54]:
# Build the endpoint URL
target_url = f"https://api.geoapify.com/v1/geocode/search?city={city}&state={state}&format=json&apiKey={api_key}"
pprint(target_url)

'https://api.geoapify.com/v1/geocode/search?city=Birmingham&state=AL&format=json&apiKey=44670ce23f39445cab010de9d5d67917'


In [55]:
import requests

filter = {} 

# Iterate through documents in the collection
for document in states.find(filter):
    # Extract "City" and "State" values from the document
    city = document['City']
    state = document['State']

    # Make API request to get latitude and longitude
    params = {
        'city': city,
        'state': state,
        'format': 'json',
        'apiKey': api_key
    }
    response = requests.get(target_url, params=params)
    data = response.json()

    # Extract latitude and longitude from the API response
    if 'results' in data and data['results']:
        lat = data['results'][0]['lat']
        lon = data['results'][0]['lon']

        # Update the document in the collection with latitude and longitude
        states.update_one({'_id': document['_id']}, {'$set': {'Lat': lat, 'Lon': lon}})

In [56]:
# review a document in the establishments collection
pprint(db.states.find_one())

{'3P': '0',
 '3PA': '0',
 'AST': '67',
 'BLK': '24',
 'Birthday': 'Feb 8, 1967',
 'City': 'Birmingham',
 'FG': '383',
 'FGA': '746',
 'FT': '260',
 'FTA': '360',
 'GP': '149',
 'Lat': {'$toDouble': '$Latitude'},
 'Lon': {'$toDouble': '$Longitude'},
 'MP': '2143',
 'ORB': '311',
 'PF': '284',
 'PTS': '1026',
 'Player': 'Michael Ansley',
 'STL': '51',
 'State': 'AL',
 'TO': '85',
 'TRB': '621',
 'Years': '3',
 '_id': ObjectId('661883863383a053ef184a23')}
